In [1]:
import numpy as np
import sys
sys.path.append('../solution')
from test_cases import TEST_CASES
from world import calc_score, MOVES, roll_coordinates, get_neighbours, apply_to_map, find_disjoint_components, update_direction
from solver import HC_1NB_CELLS, find_variants_for_mc, move_till_hp_or_death, move_till_hp_or_death2
import time

In [12]:
_map, robots = TEST_CASES[29]
print(_map)
_map = np.array([list(l) for l in _map.split('\n')])

#........L........#
#........R........#
#........L........#
#........R........#
#DUDUDUDU#UDUDUDUD#
#........R........#
#........L........#
#........R........#
#........L........#
###################


In [28]:
variants = find_variants_for_mc(_map)
hot_points = {}
for v in variants:
    hot_points[v[0]] = v[1]
arrows = {}

In [29]:
is_live = []
states = []
visited = []
for x, y, d in robots:
    states.append((x, y, d))
    visited.append(set())
    is_live.append(True)

In [30]:
def rec(x, y, d, lv, _map, visited, arrows, hot_points, depth):
    if depth == 5 or not lv:
        return None, lv, len(visited)

    sp = (x, y)
    if sp in hot_points:
        hot_point = hot_points[sp]
        del hot_points[sp]
        _max = (False, 0)
        _wmax = None
        for w in hot_point:
            arrows[sp] = w
            _x, _y, _d, _lv, _vs = move_till_hp_or_death2(x, y, d, _map, visited, arrows, hot_points)
            e1, e2, e3 = rec(_x, _y, _d, _lv, _map, visited.union(_vs), arrows, hot_points, depth + 1)
            r = (e2, e3)
            if r > _max:
                _wmax, _max = w, r
            del arrows[sp]
        hot_points[sp] = hot_point
        return _wmax, _max[0], _max[1]
    else:
        _x, _y, _d, _lv, _vs = move_till_hp_or_death2(x, y, d, _map, visited, arrows, hot_points)
        return None, rec(_x, _y, _d, _lv, _map, visited.union(_vs), arrows, hot_points, depth + 1)


In [31]:
    while any(is_live):
        for i in range(len(robots)):
            x, y, d = states[i]
            sp = (x, y)
            if sp in hot_points:
                w, _, _ = rec(x, y, d, is_live[i], _map, visited[i], arrows, hot_points, 0)                
                hot_point = hot_points[sp]
                del hot_points[sp]
                arrows[sp] = w
                x, y, d, is_live[i], new_visited_states = move_till_hp_or_death2(x, y, d, _map, visited[i], arrows, hot_points)
            else:
                x, y, d, is_live[i], new_visited_states = move_till_hp_or_death2(x, y, d, _map, visited[i], arrows, hot_points)
            visited[i] = visited[i].union(new_visited_states)
            states[i] = (x, y, d)

In [34]:
txt = ''
for e in arrows:
    if arrows[e] != '':
        txt += f'{e[0]} {e[1]} {arrows[e]} '
txt

'1 0 D 17 0 D 1 8 U 17 8 U 1 1 R 1 7 R 17 2 L 17 6 L 10 1 U 10 7 D 10 0 L 10 8 L 8 0 D 1 2 U 8 8 U 1 6 D 8 3 R 8 5 R 10 5 U 12 3 U 12 0 L 11 0 D 11 8 R 12 8 U 12 5 R 13 5 D 13 8 R 14 8 U 14 0 L 13 0 D 13 3 R 16 3 U 16 0 L 15 0 D 15 8 R 16 8 U 16 5 R 17 5 D '